In [ ]:
from keras.models import load_model, model_from_config
import h5py
import tensorflow as tf
from keras import backend as K
import shutil
import json
import numpy as np

from grpc.beta import implementations

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2

In [ ]:
K.set_learning_phase(0) 
sess = tf.Session()
K.set_session(sess)

with open('../data/zoo/07/emoji_chars.json') as fin:
    d = json.load(fin)
    emojis = d['emojis']
    char_to_idx = d['char_to_idx']
    max_sequence_len = d['max_sequence_len']

char_cnn = load_model('../data/zoo/07/char_cnn_model.h5')
config = char_cnn.get_config()
if not 'config' in config:
    config = {'config': config,
              'class_name': 'Model'}

weights = char_cnn.get_weights()
char_cnn.summary()

In [ ]:
new_model = model_from_config(config)
new_model.set_weights(weights)
new_model.compile()

In [ ]:
tweet = "There's a house centipede in my closet and since Ryan isn't here I have to kill it...."
encoded = np.zeros((1, max_sequence_len, len(char_to_idx)))
for idx, ch in enumerate(tweet):
    encoded[0, idx, char_to_idx[ch]] = 1

res = char_cnn.predict(encoded)
emojis[np.argmax(res)]

In [ ]:
input_info = tf.saved_model.utils.build_tensor_info(new_model.inputs[0])
output_info = tf.saved_model.utils.build_tensor_info(new_model.outputs[0])
prediction_signature =  tf.saved_model.signature_def_utils.build_signature_def(
          inputs={'input': input_info},
          outputs={'output': output_info},
          method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
prediction_signature

In [ ]:
# Build the signature_def_map.
classification_inputs = tf.saved_model.utils.build_tensor_info(serialized_tf_example)
classification_outputs_classes = tf.saved_model.utils.build_tensor_info(prediction_classes)
classification_outputs_scores = tf.saved_model.utils.build_tensor_info(values)

classification_signature = (
  tf.saved_model.signature_def_utils.build_signature_def(
      inputs={
          tf.saved_model.signature_constants.CLASSIFY_INPUTS:
              classification_inputs
      },
      outputs={
          tf.saved_model.signature_constants.CLASSIFY_OUTPUT_CLASSES:
              classification_outputs_classes,
          tf.saved_model.signature_constants.CLASSIFY_OUTPUT_SCORES:
              classification_outputs_scores
      },
      method_name=tf.saved_model.signature_constants.CLASSIFY_METHOD_NAME)
)

In [ ]:
outpath = '../data/zoo/07/07.2 char_cnn_model.tf_model/1'
shutil.rmtree(outpath)

legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')
builder = tf.saved_model.builder.SavedModelBuilder(outpath)
builder.add_meta_graph_and_variables(
      sess, [tf.saved_model.tag_constants.SERVING],
      signature_def_map={
           'emoji_suggest': prediction_signature,
#           signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
#               classification_signature,
      },
      legacy_init_op=legacy_init_op)
builder.save()

In [ ]:
saver = tf.train.Saver(sharded=True)
model_exporter = exporter.Exporter(saver)
signature = exporter.classification_signature(input_tensor=model.input,
                                              scores_tensor=model.output)
model_exporter.init(sess.graph.as_graph_def(),
                    default_graph_signature=signature)
model_exporter.export('zoo/07.2 char_cnn_model.tf_model', 
                      tf.constant(1), 
                      sess)

In [ ]:
request = predict_pb2.PredictRequest()
request.model_spec.name = 'char_cnn_model'
request.model_spec.signature_name = 'emoji_suggest'
request.inputs['input'].CopyFrom(
        tf.contrib.util.make_tensor_proto(encoded.astype('float32'), 
                                          shape=[1, max_sequence_len, len(char_to_idx)]))

channel = implementations.insecure_channel('localhost', 8500)
stub = prediction_service_pb2.beta_create_PredictionService_stub(channel)
result = stub.Predict(request, 5) 

In [ ]:
response = np.array(result.outputs['output'].float_val)
prediction = np.argmax(response)
emojis[prediction]